[]


In [1]:
import requests
import pandas as pd
import pyodbc
from sqlalchemy import create_engine,text
import time
from stackapi import StackAPI
import json
import sqlalchemy

In [46]:
# !git add getStackOverFlowDataToSql.ipynb
# !git reset getStackOverFlowDataToSql.ipynb
# !git checkout -b Ibrahim_branch
# !git add getStackOverFlowDataToSql.ipynb
# !git commit -m "Updated script"
# !git push origin Ibrahim_branch


# !git checkout Ibrahim_branch  
# !git pull origin Ibrahim_branch  
# !git checkout Ibrahim_branch  
# !git merge Ibrahim_branch  


To https://dev.azure.com/AmaliTech-BI/StackOverflow/_git/StackOverflow
   b44d696..6968758  Ibrahim_branch -> Ibrahim_branch


# UPDATING STACKOVERFLOW USERS

In [ ]:
from stackapi import StackAPI
from datetime import datetime, timedelta
from sqlalchemy import create_engine,text
import time
from sqlalchemy.orm import sessionmaker

# API Key
api_key = ''


# Define your connection parameters
server_name = ""
database_name = "" 
username = ""  
password = ""  
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"



# Connect to your TrendDB database using SQLAlchemy
engine = create_engine(connection_string)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

def fetch_and_append_users(last_fetched_date):
    SITE = StackAPI('stackoverflow', key=api_key)
    batch_size = 100  # Number of users per batch
    users = []
    page = 1

    while True:
        try:
            batch_start_time = time.time()

            # Fetch a batch of users with the 'fromdate' parameter
            user_data = SITE.fetch('users', page=page, pagesize=batch_size, fromdate=int(last_fetched_date.timestamp()))

            if 'items' in user_data:
                users.extend(user_data['items'])
                page += 1

                # Append fetched users to the stackoverflowUsers table in TrendDB
                df = pd.json_normalize(user_data["items"])
                print(len(df))
                df.to_sql('stackoverflowUsers', con=engine, if_exists='append', index=False)

                if 'backoff' in user_data:
                    backoff_time = user_data['backoff']
                    print(f"Throttle violation. Backing off for {backoff_time} seconds.")
                    if backoff_time > 0:
                        time.sleep(backoff_time)
                    else:
                        time.sleep(0.2)  # Introduce a small delay
                else:
                    batch_time = time.time() - batch_start_time
                    if batch_time < 1 / 30:  # 30 requests per second limit
                        wait_time = 1 / 30 - batch_time
                        print(f"Waiting for {wait_time:.2f} seconds to stay within rate limit.")
                        time.sleep(wait_time)

            else:
                break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

    # Print final number of fetched users
    print(f"Total users fetched: {len(users)}")

#
# with engine.connect() as conn:
#     conn.execute(latest_date_query)

# Get the latest date from the stackoverflowUsers table in TrendDB
latest_date_query = text("SELECT MAX(creation_date) FROM stackOverflowUsers")
latest_date = session.execute(latest_date_query).scalar()
print(latest_date)
# Fetch users from the latest date and append to the table
fetch_and_append_users(latest_date)

In [28]:
# total_users = 25000000  # Total number of users you want to fetch
# users_per_interval = 500
# requests_per_second = 30

# time_per_interval = users_per_interval / requests_per_second
# total_intervals = total_users / users_per_interval
# estimated_time = time_per_interval * total_intervals

# print(f"Estimated time to fetch all users: {estimated_time:.2f} seconds")


Estimated time to fetch all users: 833333.33 seconds


# FETCHING STACKOVERFLOW USERS

In [56]:
from stackapi import StackAPI
import time
from datetime import datetime, timedelta

# API Key
api_key = ''

SITE = StackAPI('stackoverflow', key=api_key)
batch_size = 100  # Number of users per batch
users = []
page = 1

# Calculate the date four years ago from today
four_years_ago = datetime.now() - timedelta(days=4 * 365)

while True:
    try:
        batch_start_time = time.time()

        # Fetch a batch of users with the 'fromdate' and 'todate' parameters
        user_data = SITE.fetch('users', page=page, pagesize=batch_size, fromdate=int(four_years_ago.timestamp()))

        if 'items' in user_data:
            users.extend(user_data['items'])
            page += 1

            # Display the number of fetched users so far
            print(f"Total users fetched: {len(users)}")

            if 'backoff' in user_data:
                backoff_time = user_data['backoff']
                print(f"Throttle violation. Backing off for {backoff_time} seconds.")
                time.sleep(backoff_time)
            else:
                batch_time = time.time() - batch_start_time
                if batch_time < 1 / 30:  # 30 requests per second limit
                    wait_time = 1 / 30 - batch_time
                    print(f"Waiting for {wait_time:.2f} seconds to stay within rate limit.")
                    time.sleep(wait_time)

        else:
            break

    except Exception as e:
        print(f"An error occurred: {e}")
        break

# Print final number of fetched users
print(f"Total users fetched: {len(users)}")


An error occurred: ('https://api.stackexchange.com/2.2/users/?pagesize=100&page=1&filter=default&key=bGQ5FnesrT6JD%2APvjGNP6A%28%28&fromdate=1567858688&site=stackoverflow', 502, 'throttle_violation', 'Violation of backoff parameter')
Total users fetched: 0


# FETCHING ANSWERS

In [3]:
from stackapi import StackAPI
import time

# API Key
api_key = ''


SITE = StackAPI('stackoverflow', key=api_key)
batch_size = 100  # Number of users per batch
answers = []
page = 1

while True:
    try:
        batch_start_time = time.time()

        user_data = SITE.fetch('answers', page=page, pagesize=batch_size)  # Fetch a batch of users

        if 'items' in user_data:
            answers.extend(user_data['items'])
            page += 1

            # Display the number of fetched users so far
            print(f"Total users fetched: {len(answers)}")

            if 'backoff' in user_data:
                backoff_time = user_data['backoff']
                print(f"Throttle violation. Backing off for {backoff_time} seconds.")
                if backoff_time > 0:
                    time.sleep(backoff_time)
                else:
                    time.sleep(0.1)  # Introduce a small delay
            else:
                batch_time = time.time() - batch_start_time
                if batch_time < 1 / 30:  # 30 requests per second limit
                    wait_time = 1 / 30 - batch_time
                    print(f"Waiting for {wait_time:.2f} seconds to stay within rate limit.")
                    time.sleep(wait_time)

        else:
            break

    except Exception as e:
        print(f"An error occurred: {e}")
        break

# Print final number of fetched users
print(f"Total users fetched: {len(answers)}")


Total users fetched: 500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 1000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 1500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 2000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 2500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 3000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 3500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 4000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 4500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 5000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 5500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 6000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 6500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 7000
Throttle violation. Backin

Total users fetched: 56000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 56500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 57000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 57500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 58000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 58500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 59000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 59500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 60000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 60500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 61000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 61500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 62000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 62500
Throttle vi

Total users fetched: 111500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 112000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 112500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 113000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 113500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 114000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 114500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 115000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 115500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 116000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 116500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 117000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 117500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 1180

Total users fetched: 166500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 167000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 167500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 168000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 168500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 169000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 169500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 170000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 170500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 171000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 171500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 172000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 172500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 1730

Total users fetched: 221500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 222000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 222500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 223000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 223500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 224000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 224500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 225000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 225500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 226000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 226500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 227000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 227500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 2280

...

Total users fetched: 606500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 607000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 607500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 608000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 608500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 609000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 609500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 610000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 610500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 611000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 611500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 612000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 612500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 6130

Total users fetched: 661500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 662000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 662500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 663000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 663500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 664000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 664500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 665000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 665500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 666000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 666500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 667000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 667500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 6680

Total users fetched: 716500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 717000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 717500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 718000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 718500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 719000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 719500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 720000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 720500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 721000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 721500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 722000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 722500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 7230

Total users fetched: 771500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 772000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 772500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 773000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 773500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 774000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 774500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 775000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 775500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 776000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 776500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 777000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 777500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 7780

Total users fetched: 826500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 827000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 827500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 828000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 828500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 829000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 829500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 830000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 830500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 831000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 831500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 832000
Throttle violation. Backing off for 0 seconds.
Total users fetched: 832500
Throttle violation. Backing off for 0 seconds.
Total users fetched: 8330

In [4]:
print(len(answers))

845000


In [41]:
# # Load the CSV files into dataframes
# df1 = pd.read_csv('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\users_csv_file_300m.csv')
# df2 = pd.read_csv('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\users_csv_file_100m.csv')

# # Concatenate the dataframes to combine rows
# combined_df = pd.concat([df1, df2], ignore_index=True)
# print(combined_df.duplicated().sum())

# # Remove duplicate rows
# combined_df.drop_duplicates(inplace=True)
# print(len(combined_df))

# # Save the combined and deduplicated dataframe back to a CSV file
# combined_df.to_csv('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\combined_and_deduplicated2.csv', index=False)

In [5]:
import pandas as pd
# Convert the user data to a DataFrame
df = pd.json_normalize(answers)
print(df)
df.to_csv('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\answers_csv_file_800k.csv', index=False)


        is_accepted  score  last_activity_date  last_edit_date  creation_date  \
0             False      0          1693210739    1.693211e+09     1693210686   
1             False      0          1693210738    1.693211e+09     1693209924   
2             False      0          1693210713             NaN     1693210713   
3             False      0          1693210712             NaN     1693210712   
4             False      0          1693210701             NaN     1693210701   
...             ...    ...                 ...             ...            ...   
844995        False      1          1688414745    1.688415e+09     1688409795   
844996        False      0          1688414731             NaN     1688414731   
844997         True      0          1688414677             NaN     1688414677   
844998         True      0          1688414661             NaN     1688414661   
844999         True      0          1688414609             NaN     1688414609   

        answer_id  question

In [35]:
# # print(users)
# # Convert the user data to a DataFrame
# df = pd.json_normalize(user_data["items"])
# print(df)

In [37]:
num_duplicates = df.duplicated().sum()
print(num_duplicates)

0


In [43]:
# Convert the user data to a DataFrame
df = pd.json_normalize(user_data["items"])
print(df)

     account_id  is_employee  last_modified_date  last_access_date  \
0        525299        False          1672105063        1691665765   
1       1040708        False          1688781900        1426924682   
2       1152914        False          1685755800        1692216999   
3       1194198        False          1682128500        1690615542   
4        515885        False          1615937700        1692651266   
..          ...          ...                 ...               ...   
495     5962976        False          1654913100        1685270567   
496     7004951        False          1683777740        1692739779   
497     8318014        False          1674465600        1692779281   
498    13441541        False          1644217074        1692772811   
499    15967589        False          1689262510        1692757385   

     reputation_change_year  reputation_change_quarter  \
0                        79                         20   
1                         0                

In [145]:
# # Identify columns with dictionary data
# dict_columns = [col for col, dtype in df.dtypes.items() if dtype == "object" and isinstance(df[col][0], dict)]


# # Convert dictionary columns to JSON strings
# for col in dict_columns:
#     df[col] = df[col].apply(json.dumps)
    
# print(df.columns)


In [ ]:
df.to_csv('C:\Users\IbrahimOsuman\Desktop\Tabular\users_csv_file', index=False)

In [64]:
# Convert user data to a DataFrame for observation
df = pd.DataFrame(user_data)
print(len(df))
print(df.head())  # Observe the first 5 entries

1000
                                    badge_counts  \
0  {'bronze': 9190, 'silver': 9123, 'gold': 867}   
1  {'bronze': 5234, 'silver': 4402, 'gold': 529}   
2     {'bronze': 783, 'silver': 645, 'gold': 58}   
3  {'bronze': 3555, 'silver': 3610, 'gold': 371}   
4  {'bronze': 3342, 'silver': 4053, 'gold': 295}   

                                         collectives  account_id  is_employee  \
0  [{'collective': {'tags': ['firebase-performanc...       11683        False   
1  [{'collective': {'tags': ['tibble', 'quantmod'...        4243        False   
2                                                NaN     1165580        False   
3                                                NaN       52822        False   
4                                                NaN       35417        False   

   last_modified_date  last_access_date  reputation_change_year  \
0          1691840115        1692608062                   41934   
1          1691892604        1692609261                   771

In [3]:
# import requests

# # Your personal access token
# token = ""

# # GraphQL endpoint
# graphql_endpoint = "https://api.github.com/graphql"

# # Your GraphQL query
# query = """
# {
#   search(query: "is:public", type: REPOSITORY, first: 100) {
#     repositoryCount
#     edges {
#       node {
#         ... on Repository {
#           name
#           primaryLanguage {
#             name
#           }
#           languages_url
#           owner {
#             login
#           }
#           subscribers_count
#           repositoryTopics(first: 10) {
#             edges {
#               node {
#                 topic {
#                   name
#                 }
#               }
#             }
#           }
#           watchers {
#             totalCount
#           }
#           nameWithOwner
#           tagsUrl
#           createdAt
#           collaborators {
#             totalCount
#           }
#           hasProjects
#           # Add more fields as needed
#         }
#       }
#     }
#     pageInfo {
#       hasNextPage
#       endCursor
#     }
#   }
# }
# """

# # Initialize variables for pagination
# hasNextPage = True
# endCursor = ""

# repo_data = []

# while hasNextPage:
#     headers = {
#         "Authorization": f"Bearer {token}",
#         "Accept": "application/vnd.github.v4+json"
#     }
    
#     # Fetch the page of results using the query and cursor
#     result = requests.post(graphql_endpoint, json={"query": query}, headers=headers)
#     data = result.json()
    
#     search_data = data.get('data', {}).get('search', {})
#     repositories = search_data.get('edges', [])
#     pageInfo = search_data.get('pageInfo', {})
    
#     # Process the repositories
#     for repo in repositories:
#         repo_info = {
#                 'name': repo['node']['name'],
#                 'language': repo['node']['primaryLanguage']['name'] if repo['node']['primaryLanguage'] else None,
#                 'languages_url': repo['node']['languages_url'],
#                 'organization': repo['node']['owner']['login'],
#                 'subscribers_count': repo['node']['subscribers_count'],
#                 'topics': [topic['node']['topic']['name'] for topic in repo['node']['repositoryTopics']['edges']],
#                 'watchers_count': repo['node']['watchers']['totalCount'],
#                 'full_name': repo['node']['nameWithOwner'],
#                 'tags_url': repo['node']['tagsUrl'],
#                 'created_at': repo['node']['createdAt'],
#                 'contributors_url': f"https://api.github.com/repos/{repo['node']['owner']['login']}/{repo['node']['name']}/contributors",
#                 'collaborators_url': f"https://api.github.com/repos/{repo['node']['owner']['login']}/{repo['node']['name']}/collaborators",
#                 'has_projects': repo['node']['hasProjects']
#             }
#         repo_data.append(repo_info)
    
#     hasNextPage = pageInfo.get('hasNextPage', False)
#     endCursor = pageInfo.get('endCursor', None)

# # Now, repo_data should contain information for all repositories
# print(f"Total repositories fetched: {len(repo_data)}")


Total repositories fetched: 0


In [26]:
# # Writing the data to the CSV file
# with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
#     fieldnames = ['display_name']
#     writer = csv.DictWriter(file, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerows(user_data)

# print("CSV file saved successfully.")

In [58]:
# import csv


# csv_file_path = "C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch3.csv"

# # Extract the keys for the CSV header
# csv_columns = user_data[0].keys()

# # Writing the data to the CSV file
# with open(csv_file_path, mode='w', newline='',encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=csv_columns)
#     writer.writeheader()
#     writer.writerows(user_data)

# print("CSV file saved successfully.")


CSV file saved successfully.


In [18]:
# import csv

# def count_duplicates(csv_file_path):
#     seen_data = set()
#     duplicate_count = 0

#     with open(csv_file_path, mode='r', newline='') as file:
#         reader = csv.reader(file)
#         header = next(reader)  # Skip the header

#         for row in reader:
#             row_tuple = tuple(row)
            
#             if row_tuple in seen_data:
#                 duplicate_count += 1
#             else:
#                 seen_data.add(row_tuple)

#     return duplicate_count

# # csv_file_path = "your_csv_file.csv"
# duplicate_count = count_duplicates(csv_file_path)

# print(f"Number of duplicate rows: {duplicate_count}")


Number of duplicate rows: 0


In [36]:
print(user_data))

4026


In [72]:
# import csv


# csv_file_path = "C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch6.csv"

# # Extract the keys for the CSV header
# csv_columns = user_data[0].keys()

# # Writing the data to the CSV file
# with open(csv_file_path, mode='w', newline='',encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=csv_columns)
#     writer.writeheader()
#     writer.writerows(user_data)

# print("CSV file saved successfully.")

CSV file saved successfully.


In [73]:
# import pandas as pd

# # List of CSV file paths to merge
# file_paths = ['C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch1.csv', 
# 'C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch2.csv',
#  'C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch3.csv',
#  'C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch4.csv',
#   'C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch5.csv',
#   'C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\repos_fetched_batch6.csv'
#   ]

# # Initialize an empty DataFrame to store the merged data
# merged_data = pd.DataFrame()

# # Loop through the file paths and merge the CSV data
# for file_path in file_paths:
#     data = pd.read_csv(file_path)
#     merged_data = pd.concat([merged_data, data], ignore_index=True)
# print(len(merged_data))

# # Save the merged data to a new CSV file
# merged_data.to_csv('Ibrahimmerged_Repodata.csv', index=False)

# print("CSV files merged and saved as merged_data.csv")

25853
CSV files merged and saved as merged_data.csv


In [74]:
# file_paths = ['Ibrahimmerged_Repodata.csv',
#   'G:\\Shared drives\\IGUS Project\\PowerBI Projects\\repos_fetched_Rebecca.csv'
#   ]

# # Initialize an empty DataFrame to store the merged data
# merged_data = pd.DataFrame()

# # Loop through the file paths and merge the CSV data
# for file_path in file_paths:
#     data = pd.read_csv(file_path)
#     merged_data = pd.concat([merged_data, data], ignore_index=True)
# print(len(merged_data))

# # Save the merged data to a new CSV file
# merged_data.to_csv('G:\\Shared drives\\IGUS Project\\PowerBI Projects\\Repositories.csv', index=False)

# print("CSV files merged and saved as merged_data.csv")

53956
CSV files merged and saved as merged_data.csv


In [30]:
# import csv

# # Read data from Githubusers.csv
# github_users = set()
# with open('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\GithubUsers_1020.csv', mode='r', newline='', encoding='utf-8') as github_file:
#     github_reader = csv.DictReader(github_file)
#     for row in github_reader:
#         github_users.add(row['username'])
#         github_users.add(row['name'])

# # Compare and write data to usersTobeSearchedFor.csv
# users_to_search_for = set()
# with open('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\users_csv_file_100m.csv', mode='r', newline='', encoding='utf-8') as stackoverflow_file:
#     stackoverflow_reader = csv.DictReader(stackoverflow_file)
#     for row in stackoverflow_reader:
#         display_name = row['display_name']
#         if display_name in github_users:
#             users_to_search_for.add(display_name)

# with open('usersTobeSearchedFor.csv', mode='w', newline='', encoding='utf-8') as output_file:
#     fieldnames = ['display_name']
#     writer = csv.DictWriter(output_file, fieldnames=fieldnames)
#     writer.writeheader()
#     for display_name in users_to_search_for:
#         writer.writerow({'display_name': display_name})

# print("Done! usersTobeSearchedFor.csv created.")


Done! usersTobeSearchedFor.csv created.


In [44]:
# import csv

# # Read data from Githubusers.csv
# github_users = set()
# with open('G:\\Shared drives\\IGUS Project\\PowerBI Projects\\github users500k.csv', mode='r', newline='', encoding='utf-8') as github_file:
#     github_reader = csv.DictReader(github_file)
#     for row in github_reader:
#         github_users.add(row['login'])
# #         github_users.add(row['name'])

# # Compare and write data to usersTobeSearchedFor.csv
# users_to_search_for = set()
# with open('C:\\Users\\IbrahimOsuman\\Desktop\\Tabular\\users_csv_file_100m.csv', mode='r', newline='', encoding='utf-8') as stackoverflow_file:
#     stackoverflow_reader = csv.DictReader(stackoverflow_file)
#     for row in stackoverflow_reader:
#         display_name = row['display_name']
#         if display_name in github_users:
#             users_to_search_for.add(display_name)

# with open('usersTobeSearchedFor.csv', mode='w', newline='', encoding='utf-8') as output_file:
#     fieldnames = ['display_name']
#     writer = csv.DictWriter(output_file, fieldnames=fieldnames)
#     writer.writeheader()
#     for display_name in users_to_search_for:
#         writer.writerow({'display_name': display_name})

# print("Done! usersTobeSearchedFor2.csv created.")

Done! usersTobeSearchedFor2.csv created.
